This notebook provides examples of how to load, examine, and visualize electrophysiological data containing known cell types. This data comes from the Cell Type Lookup Table dataset, which contains **extracallular electrophysiological** data with specific cell types identified via **optotagging**.

In [1]:
from hdmf_zarr import NWBZarrIO
import os

Unlike other Allen Institute datasets, these data are stored as NWBs, with the metadata stored as jsons. We will be using hdmf_zarr to interact with this dataset.

## Load an NWB

Let's try loading and exploring one session's worth of data.

In [2]:
# an 'arbitrarily' selected session
session = '661398_2023-04-03_15-47-29'
session_directory = f'/data/SWDB 2024 CTLUT data/ecephys_{session}_nwb'

nwb_file = os.path.join(session_directory, f'ecephys_{session}_experiment1_recording1.nwb.zarr')
io = NWBZarrIO(nwb_file, "r")
nwbfile_read = io.read()

You can use the **nwbwidgets** module to visually explore the contents of an nwb file.

In [3]:
from nwbwidgets import nwb2widget
nwb2widget(nwbfile_read)

ModuleNotFoundError: No module named 'nwbwidgets'

## Load the units table

The units table contains all the "neurons" that were detected by the spike sorter (in this case, Kilosort 2.5). We usually refer to them as "units" because we can't actually be completely certain that the spike sorter output corresponds 1-to-1 to the actual neurons that were present at the time of the recording: this is just our best guess!

In [ ]:
units = nwbfile_read.units[:]
units.head()

In [ ]:
units.columns

You may notice this units table has a lot of columns! Many of these are quality metrics or laser response metrics, which are more fully explained in the databook. You can use these to filter out units that are unlikely to be real neurons or units whose data is too noisy for analysis. Among the most interesting columns is the **spike_times** column, which tells us every time point when this "neuron" fired an "action potential."

Let's try plotting all the units in a raster plot, to see what activity looked like prior to the laser stimulation!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

start_time = nwbfile_read.epochs.start_time[0]
stop_time = nwbfile_read.epochs.stop_time[0]

for unit_id, unit_row in units.iterrows():
    unit_spiketimes = unit_row.spike_times
    pre_stim_spiketimes = unit_spiketimes[unit_spiketimes<stop_time]
    plt.plot(pre_stim_spiketimes, np.tile(unit_id, len(pre_stim_spiketimes)), 'k.', ms=1, alpha=0.1)
    
plt.xlabel('Time (s)')
plt.ylabel('Unit ID')

However, a lot of these "neurons" are probably noise or artifacts. We can use some of the quality metrics contained in the units table to filter out some of the "bad" "neurons." A common way to curate units is by the amount of ISI (interspike interval) violations. Neurons physiologically can not spike more often than every 2 ms, so a high ratio of "spikes" that happen less than 2 ms apart is a good indicator that this unit is not a real neuron (or, at least, not a *single* neuron).

Another way units are curated is by visually inspecting the spike waveforms to determine if they look like action potentials or just electrical noise. This table contains the results on someone's manual curation of the waveforms in the column *noise_label*.

In [ ]:
# query the units table to get units with good waveforms and low ISI violations!
good_units = units.query("isi_violations_ratio < 0.5 and noise_label == 'good'")

# and now plot only the "good" units
for unit_id, unit_row in good_units.iterrows():
    unit_spiketimes = unit_row.spike_times
    pre_stim_spiketimes = unit_spiketimes[unit_spiketimes<stop_time]
    plt.plot(pre_stim_spiketimes, np.tile(unit_id, len(pre_stim_spiketimes)), 'k.', ms=1, alpha=0.1)
    
plt.xlabel('Time (s)')
plt.ylabel('Unit ID')

**Task 1**: The most common standard QC criteria used at the Allen Institute are isi_violations_ratio < 0.5, amplitude_cutoff < 0.1, and presense_ratio > 0.8. Try querying for all three of these criteria and plotting the resulting *really* good units!

Many of these nwb files also contain units from multiple probes concatenated together. Let's see how many probes were used in this recording.

In [ ]:
probes = np.unique(units.device_name)
probes

It might be more useful to have separate plots for the two probes.

In [ ]:
for ind_probe, probe in enumerate(probes):
    plt.subplot(1,2,ind_probe+1)
    this_probe_units = good_units.query("device_name == @probe")
    
    for unit_id, unit_row in this_probe_units.iterrows():
        unit_spiketimes = unit_row.spike_times
        pre_stim_spiketimes = unit_spiketimes[unit_spiketimes<stop_time]
        plt.plot(pre_stim_spiketimes, np.tile(unit_id, len(pre_stim_spiketimes)), 'k.', ms=1, alpha=0.1)
        plt.title(probe)

**Task 2**: The *peak_channel* column contains the electrode closest to the unit, as determined by the peak waveform. This gives us a location for the unit. Try plotting the above rasters with *peak_channel* as the y-axis rather than *unit_id*, which is assigned somewhat arbitrarily!

## Identify tagged units

During this experiment, known cell types were identified by their responses to pulses of laser. You can learn more about this technique in the databook page on Optotagging! The units table contains laser response metrics for every units, as well as a predicted cell type label. Let's try filtering the units table by cell type!

In [ ]:
np.unique(units.predicted_cell_type)

This experiment was tagging D1 and D2 medium spiny neurons in the striatum! We can filter the units table for units for which we have a predicted cell type. Note that just because a unit is labeled as "untagged," that does not guarantee it *isn't* a D1 or D2 cell. We just don't know what it is.

In [ ]:
tagged_units = units.query("predicted_cell_type != 'untagged'")
tagged_units.head()

Let's see what the activity of just these units looks like!

In [ ]:
cell_type_colors = {'D1' : 'skyblue', 'D2' : 'tomato'}

for ind, (unit_id, unit_row) in enumerate(tagged_units.iterrows()):
    unit_spiketimes = unit_row.spike_times
    pre_stim_spiketimes = unit_spiketimes[unit_spiketimes<stop_time]

    # let's try making a psth of the response instead of a raster!
    bin_length = 5
    time_bins = np.arange(np.floor(start_time),np.ceil(stop_time),bin_length)
    h,v = np.histogram(unit_spiketimes, time_bins)
    plt.plot(v[:-1],h/bin_length, color=cell_type_colors[unit_row.predicted_cell_type])
    
plt.xlabel('Time (s)')
plt.ylabel('Firing rate (spikes/s)')

**Task 3**: cell type predictions are based only on the unit's laser reponses, not the quality of the unit. There's a good chance many of these are laser artifacts or other types of noise or multiunit activity. Try further filtering the units table for tagged, *quality* units.

## Load running speed

Some behavioral data was collected during this experiment in addition to the physiology. For instance, the mouse was on a running wheel with an encoder, and so we recorded the mouse's running speed for the entire experiment. Let's try loading that data!

In [ ]:
running = nwbfile_read.processing['behavior']['BehavioralTimeSeries']['linear velocity']
running_speed = np.array(running.data)
running_timestamps = np.array(running.timestamps)

In [ ]:
# plot the running speed for the same time period as the rasters above
pre_stim_timestamps = running_timestamps[running_timestamps < stop_time]
pre_stim_speed = running_speed[running_timestamps < stop_time]

plt.plot(pre_stim_timestamps, pre_stim_speed)

**Task 4**: Try plotting the running speed and the units raster in the same figure with a shared x-axis. Does any of the neuronal activity seem related to the mouse's movement?